In [2]:
import os
import re
import math
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

from matplotlib.pyplot import figure


import networkx as nx
import seaborn as sns

from wordcloud import WordCloud
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import re
import string
import math
import random
from random import choice, choices
import time
import gc
from IPython.display import display

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from scipy.sparse import coo_matrix
# import implicit

import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    

import os
import gensim
from nltk.tokenize import word_tokenize
import dateutil.parser
from datetime import datetime
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)

%matplotlib inline


[nltk_data] Downloading package punkt to /Users/lin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#pro= pd.read_csv('professionals.csv')
#stu=pd.read_csv("students.csv")
qs=pd.read_csv("questions.csv")
ans= pd.read_csv('answers.csv')
#com= pd.read_csv('comments.csv')
#email= pd.read_csv('emails.csv')
#match= pd.read_csv('matches.csv')
#school_mem= pd.read_csv('school_memberships.csv')
tags= pd.read_csv('tags.csv')
qs_tags=pd.read_csv('tag_questions.csv')
#user_tag=pd.read_csv('tag_users.csv')
occupation = pd.read_csv('occupation_corpus.csv')

## Build out Recommender - Find Similar Questions

Find similar questions that professionals have answered in the past so students don't need to post same questions. 

In [4]:
# Concentrate tags to questions

qs_tagnames= pd.merge(qs_tags,tags,left_on='tag_questions_tag_id',right_on='tags_tag_id')
qs_tagnames=qs_tagnames.drop(['tags_tag_id','tag_questions_tag_id'],axis=1)

qs_tag_pivot=qs_tagnames.pivot_table(index='tag_questions_question_id',values='tags_tag_name',aggfunc=lambda x: " ".join(x))
qs_tag_pivot['tag_questions_question_id']=qs_tag_pivot.index

qs_tag_pivot=qs_tag_pivot.reset_index(drop=True)

# Combine question table with tags

qs_with_tags=pd.merge(qs,qs_tag_pivot,left_on='questions_id',right_on='tag_questions_question_id')

# Combine question title, body, and tags

raw_documents=qs_with_tags['questions_title']+qs_with_tags['questions_body']+qs_with_tags['tags_tag_name']



### Text Processing

In [5]:
# print("Number of Questions:",len(raw_documents))

#Tokenizing data
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]

# Create dictionary
dictionary = gensim.corpora.Dictionary(gen_docs)

# Creat Document-Term Matrix
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

# Creat TF-IDF Model
tf_idf = gensim.models.TfidfModel(corpus)

# Creat Similarity Checker
similar_qs = gensim.similarities.Similarity("",tf_idf[corpus],num_features=len(dictionary))

#print("Processing Completed!")

In [6]:
def recommendation(query):
    query_doc = [w.lower() for w in word_tokenize(query)]
    query_doc_bow = dictionary.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    q_sim=similar_qs[query_doc_tf_idf]
    sim_threshold=0.10
    
    # Display similar questions from the past:

    qs_with_tags['Similarity']=q_sim
    ques=qs_with_tags[qs_with_tags['Similarity']>=sim_threshold]
    ques=ques.sort_values('Similarity',ascending=False)
    ques = ques[['questions_title', 'questions_body']]
    return ques.head()


In [42]:
qs['questions_title'][0:5]

0    Teacher   career   question                                                                
1    I want to become an army officer. What can I do to become an army officer?                 
2    Will going abroad for your first job increase your chances for jobs back home?             
3    To become a specialist in business  management, will I have to network myself?             
4    Are there any scholarships out there for students that are first generation and live in GA?
Name: questions_title, dtype: object

In [43]:
result = {}

for question in qs['questions_title']:
    key = question
    result[key] = recommendation(key)


In [52]:
# dict = {'Python' : '.py', 'C++' : '.cpp', 'Java' : '.java'}
f = open("dict.txt","w")
f.write( str(result) )
f.close()
